In [1]:
import requests # from urllib.request import urlopen
from bs4 import BeautifulSoup # HTTP Response -> HTML 

import pandas as pd

from datetime import datetime

import time 
import re #정규 표현식 활용

In [7]:
#망고플레이트에서 음식을 키워드로 맛집 리스트 가져오는 function
def get_rest_list(keyword):
    
    #페이지, 맛집리스트 초기화
    pagenum = 1
    restaurant_list = []
    
    #page별로 while 루프 돌려서 상호명 뽑아오기
    while pagenum <= 10: 
        try:
            url = 'https://www.mangoplate.com/search/?keyword='+keyword+'&page='+str(pagenum)#keyword를 input으로 망고플레이트 url 생성
            headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
            }

            with requests.Session() as session:
            
                html = session.get(url, headers=headers).content 
                source = BeautifulSoup(html, 'html.parser')

            restaurants = source.find_all('h2', {'class' : 'title'}) #h2태그의 class 'title' => 상호명

            for restaurant in restaurants:#텍스트 깔끔하게 처리 및 리스트에 상호명 추가 루프

                n_restaurant = re.sub('\s', '', restaurant.get_text()) #텍스트 중간에 빈칸 없애기
                restaurant_list.append(n_restaurant)#리스트에 상호명 추가
        except:
            print(pagenum,'페이지 까지의 결과입니다.')
        
        pagenum += 1
        
    return restaurant_list #keyword 관련 맛집 리스트 생성

In [8]:
currylist = get_rest_list('카레')

In [9]:
currylist

['마루심',
 '우미노미',
 '카레',
 '공기식당',
 '요시즈카우나기야/吉塚うなぎ屋',
 '마루심(마포점)',
 '멘야하나비(신사점)',
 '예티',
 '고옥',
 '홋카이도부타동스미레',
 '데카망(휴업중)',
 '요우란',
 '온정',
 '타이가텐푸라',
 '오사카카레콘유',
 '텐동요츠야',
 '멘야하나비',
 '깔리',
 '우마텐',
 '부타이',
 '에베레스트(동대문점)',
 '가츠오',
 '도요스6가',
 '텐동한',
 '현우동',
 '라하노카레',
 '포탈라(종로점)',
 '후라토식당(아브뉴프랑판교점)',
 '운멜로랩',
 '이치젠(망원본점)',
 'BARN110',
 '후라토식당',
 '오복수산(도산점)',
 '교양식사',
 '멘야코노하카레',
 '사쿠텐',
 '이코이',
 '스아게/すあげ',
 '주간소바식당산',
 '하이쿠',
 '온돈부리',
 '어제의카레',
 'The92산들바다',
 '비나',
 '커리숲',
 '킨이로텐',
 '고가빈커리하우스',
 '툭툭누들타이',
 '우동명가기리야마',
 '종로돈부리',
 '히메시야',
 '옷살',
 '카린지',
 '형제돈부리',
 '수퍼판',
 '우동카덴(연희점)',
 '미쁘동',
 '카타코토',
 '기요한',
 '바그다드',
 '곳',
 '피망과토마토',
 '4.5평우동집',
 '정돈(홍대점)',
 '日本橋海鮮丼つじ半(츠지한)',
 '야스노야(신용산점)',
 '쉐맘',
 '맛의정원',
 '케루악',
 '동경밥상',
 '라이프카레앤커피',
 '온센(신당점)',
 '유메',
 '카레클린트(분당용인점)',
 '맘스키친',
 '죠티레스토랑',
 '소년식당',
 '마구로쇼쿠도',
 '오복수산',
 '호랑이카레',
 '돈부리',
 '마이클식당',
 '스아게K',
 '토마',
 '오자와',
 '온미동',
 '유야케도쿄',
 '여도가주',
 '고로케야',
 '야마키치',
 '밥플러스',
 '비스트로큐슈',
 '다케',
 '색다른면',
 '나지미돈부리',
 '아그라(에비뉴종로점)',
 '카레바시야',
 '겨리'

In [10]:
def get_mango_info(restaurant):#상호명을 input으로 관련 정보 받아오는 function

    mango_info = {} #빈 dict 생성

    url = 'https://www.mangoplate.com/search/'+restaurant #상호명으로 망고플레이트 검색 url

    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    
    with requests.Session() as session:
            
                html = session.get(url, headers=headers).content 
                source = BeautifulSoup(html, 'html.parser')
    
    try:
        rest_code = source.find('div',{'class':'list-restaurant-item'}).find('a')['href']#검색결과에서 첫번째 결과의 고유 코드 따오기 
        newurl = 'https://www.mangoplate.com'+rest_code #고유 코드 활용한 url 주소 생성
        newhtml = session.get(newurl, headers=headers).content 
        newsource = BeautifulSoup(newhtml, 'html.parser')

        rate = float(re.sub('\s','',newsource.find('strong',{'class':'rate-point'}).get_text()))#평점이 속한 태그에서 텍스트 가져와 float로 바꾸기 (나중에 계산 용이)

        rest_table = newsource.find('table',{'class':'info'})#관련 정보가 있는 테이블 경로

        address = rest_table.find_all('td')[0].find_all('span')[1].get_text()#주소
        phone = rest_table.find_all('td')[1].get_text()#전화번호
        kind = re.sub('\s','',rest_table.find_all('td')[2].get_text())#음식 종류(정규표현식 활용하여 깔끔하게 정리)
        price = rest_table.find_all('td')[3].get_text()#가격대
        
        #리뷰수
        index_num = newsource.find('script',{'type':'application/ld+json'} ).get_text().find('reviewCount') + 13
        reviewcount = int(newsource.find('script',{'type':'application/ld+json'} ).get_text()[index_num:index_num + 2])

        #각각의 정보를 만든 dict의 key값의 value로 추가
        mango_info['name'] = restaurant
        mango_info['rate'] = rate
        mango_info['address'] = address
        mango_info['phone'] = phone
        mango_info['kind'] = kind
        mango_info['price'] = price
        mango_info['reviewcount'] = reviewcount
        
    except:
        mango_info['name'] = restaurant
        mango_info['rate'] = 'N/A'
        mango_info['address'] = 'N/A'
        mango_info['phone'] = 'N/A'
        mango_info['kind'] = 'N/A'
        mango_info['price'] = 'N/A'
        mango_info['reviewcount'] = 'N/A'
    
    return mango_info #하나의 dict를 최종 값으로 return



In [11]:
print(get_mango_info('윤밀원'))

{'name': '윤밀원', 'rate': 4.6, 'address': '경기도 성남시 분당구 정자동 223-9', 'phone': '031-714-8388', 'kind': '국수/면요리', 'price': '만원 미만', 'reviewcount': 55}


In [12]:
def get_dining_info(restaurant): #다이닝코드 평점/별점/리뷰 가져오기
    
    dining_info = {}
    review_list = []
    
    #정규표현식 활용해 상호명에 있는 괄호 및 특수문자 제거
    regex = "\(.*\)|\s-\s.*"
    query = re.sub(regex, '', restaurant)
    
    url = 'https://www.diningcode.com/isearch.php?query=' + query
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')
    
    #다이닝코드에 해당 식당이 없는 경우를 예상하여 try/except로 error 발생 시 'N/A'로 데이터 대체
    try:
        ul_tag = source.find('div', {'id':'div_rn'})
        li_tag = ul_tag.find('li')
        a_tag = li_tag.find('a')
        target_url = 'https://www.diningcode.com' + a_tag.attrs['href']
        target_url

        web_review = requests.get(target_url).content
        source_review = BeautifulSoup(web_review, 'html.parser')

        #평균 평점 꺼내기
        p_tag = source_review.find('p', {'class':'grade'})
        # p_tag
        review_point = int(p_tag.find('strong').get_text().replace('점',''))

        #평균 별점 꺼내기
        star_point = source_review.find('p', {'id':'lbl_star_point'}).get_text()
        star_point = star_point.replace("\n", "")
        star_point = float(star_point.replace('점',''))# 교체하다

        reviews = source_review.find_all('p',{'class':'review_contents btxt'})

        for review in reviews:
            review_list.append(re.sub('\n','',review.get_text()))

        dining_info['review_point'] = review_point
        dining_info['star_point'] = star_point
        dining_info['review'] = review_list
                               
    except:
        dining_info['review_point'] = 'N/A'
        dining_info['star_point'] = 'N/A'
        dining_info['review'] = 'N/A'
        
    return dining_info

In [13]:
print(get_dining_info('윤밀원'))

{'review_point': 67, 'star_point': 4.7, 'review': ['한글날 딸들을 데리고 분당 정자동 윤밀원으로  평양냉면 양곰탕 양지국수를 주문 평양냉면 면발 매밀함량 굿 식감 굿 육수는 동치미맛이 짙음 양지국수 먹어보지 못했던 특색있는 깊고 칼칼한 맛 굿 양 곰탕은 고기냄새는잘 잡았으나 국물은 그다지 깨끗하게 개운하지 못했음 다음엔 족발을 꼭 먹어봐야겠다 주택가에 위치해 가로수들이 단풍이들어 가을 정취를 도심에서 느낄수있어 더더욱 좋았던 가족나들이였습니다', '족발은 요즘 유행하는 스타일이고 제공되는 고수와 소스등으로 깔끔함. 평양냉면 역시 입문자가 먹어도 무난한 맛. 칼국수는 개인적으로 별로였습니다. 대기가 길지만 일단 입장하면 음식도 빨리 나오고 서비스도 좋아서 기분 좋게 식사 가능합니다. 주차는 근처 공영 주차장 추천합니다.', '첫번째 사진 : 양지칼국수두번째 사진 : 막국수세번째 사진 : 냉면칼국수 종류(양지칼국수, 칼국수)는 흰국물과 빨간국물 두가지 종류가 있다고 한다. 양지칼국수 흰국물로 시켰는데 순하고 담백한 맛이었다.막국수는 막국수보단 비빔냉면 같은 느낌이었다. 자극적이지 않으며 얘 또한 순하고 단백한 맛이었다.냉면은 평양냉면 스타일같고, 순한 맛이었다.총평 : 멀리갈게 아니라면 아주 괜찮은 선택지가 될거같다. 재방문의사 있음.양이 상당히 많다. 대식가가 아니라면 인원수보다 적게 시키는데 좋을거같으며, 족발을 시키는 경우에는 더 적게 시켜도 될거 같다. 전체적으로 고급지고 단백하고 순한 맛으로 어른들이 좋아할것 같고, 아이들도 칼국수와 막국수는 좋아할것 같다. 메뉴가 전체적으로 순한맛이라 막국수도 자극적이지 않아서 어린 아이들이 먹기 좋을거같다. 재방문의사 있음.', '근처에 공영 주차장이 있어서 거기에 차 대시면 됩니다. 막국수가 진짜 부드럽게 녹아요. 맛있어요. 족발도 잡내 없고 정말 맛있네요.', '나이와 상관없이 맛난 음식을 즐길수있는 행복한 장소']}


In [14]:
#음식을 키워드로 해당 음식 맛집리스트를 뽑아오는 function
def get_restaurant_data(menu):
    
    #만들어놓은 get_rest_list function으로 상호명 리스트 생성
    restaurant_list = get_rest_list(menu)
    print(len(restaurant_list),'개의 식당 검색 완료.')
    
    #DataFrame을 위한 빈 리스트 생성

    mango_name = []
    mango_rate = []
    mango_address = []
    mango_phone = []
    mango_kind = []
    mango_price = []
    mango_reviewcount = []
    dining_review_point = []
    dining_star_point = []
    dining_review = []
    count = 1

    #상호명 리스트를 활용하여 각각의 점포의 정보를 망고플레이트, 네이버, 다이닝코드로 부터 받아오는 for loop
    for restaurant in restaurant_list:

        mango_info = get_mango_info(restaurant)
        dining_info = get_dining_info(restaurant)
        #위에서 정의한 망고플레이트 음식점 정보를 하나의 dict로 가져옴

        #각각의 리스트에 고유 정보 추가
        mango_name.append(mango_info['name'])
        mango_rate.append(mango_info['rate'])
        mango_address.append(mango_info['address'])
        mango_phone.append(mango_info['phone'])
        mango_kind.append(mango_info['kind'])
        mango_price.append(mango_info['price'])
        mango_reviewcount.append(mango_info['reviewcount'])
        dining_review_point.append(dining_info['review_point'])
        dining_star_point.append(dining_info['star_point'])
        dining_review.append(dining_info['review'])
        
        print(count,'번째 식당 정보 수집중...')
        count += 1
    
    #생성된 리스트로 pandas dataframe 생성
    restaurant_df = pd.DataFrame({'음식점':mango_name,
                         '망고평점': mango_rate,
                         '망고리뷰수': mango_reviewcount,
                         '다이닝코드평점': dining_review_point,
                         '다이닝코드별점': dining_star_point,
                         '다이닝코드리뷰': dining_review,     
                         '주소': mango_address,
                         '전화': mango_phone,
                         '종류': mango_kind,
                         '가격': mango_price})
    
    #완성된 DataFrame으로 excel파일 생성
    restaurant_df.to_excel('{}맛집리스트_{}.xlsx'.format(menu,datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')
    print(menu,'맛집리스트가 완성되었습니다.')
    print(restaurant_df.head())
    
    #완성된 Dataframe을 return
    return restaurant_df
    

In [15]:
pork_cutlet_df = get_restaurant_data('돈까스')

200 개의 식당 검색 완료.
1 번째 식당 정보 수집중...
2 번째 식당 정보 수집중...
3 번째 식당 정보 수집중...
4 번째 식당 정보 수집중...
5 번째 식당 정보 수집중...
6 번째 식당 정보 수집중...
7 번째 식당 정보 수집중...
8 번째 식당 정보 수집중...
9 번째 식당 정보 수집중...
10 번째 식당 정보 수집중...
11 번째 식당 정보 수집중...
12 번째 식당 정보 수집중...
13 번째 식당 정보 수집중...
14 번째 식당 정보 수집중...
15 번째 식당 정보 수집중...
16 번째 식당 정보 수집중...
17 번째 식당 정보 수집중...
18 번째 식당 정보 수집중...
19 번째 식당 정보 수집중...
20 번째 식당 정보 수집중...
21 번째 식당 정보 수집중...
22 번째 식당 정보 수집중...
23 번째 식당 정보 수집중...
24 번째 식당 정보 수집중...
25 번째 식당 정보 수집중...
26 번째 식당 정보 수집중...
27 번째 식당 정보 수집중...
28 번째 식당 정보 수집중...
29 번째 식당 정보 수집중...
30 번째 식당 정보 수집중...
31 번째 식당 정보 수집중...
32 번째 식당 정보 수집중...
33 번째 식당 정보 수집중...
34 번째 식당 정보 수집중...
35 번째 식당 정보 수집중...
36 번째 식당 정보 수집중...
37 번째 식당 정보 수집중...
38 번째 식당 정보 수집중...
39 번째 식당 정보 수집중...
40 번째 식당 정보 수집중...
41 번째 식당 정보 수집중...
42 번째 식당 정보 수집중...
43 번째 식당 정보 수집중...
44 번째 식당 정보 수집중...
45 번째 식당 정보 수집중...
46 번째 식당 정보 수집중...
47 번째 식당 정보 수집중...
48 번째 식당 정보 수집중...
49 번째 식당 정보 수집중...
50 번째 식당 정보 수집중...
51 번째 식당 정보 수집중...
52 번째 식당 정보 수집중...
53 번

In [13]:
pork_cutlet_df['음식점']

0                  시키카츠
1                  카와카츠
2                   오제제
3                카츠바이콘반
4      후니도니(르메이에르종로타운점)
5                    북천
6                  시키카츠
7                  드림카츠
8                크레이지카츠
9               망원동즉석우동
10                돈까스전원
11                 얌얌카츠
12                   리애
13              햇살머무는식탁
14               정돈프리미엄
15             정돈(강남역점)
16                  부타이
17               최강금돈까스
18                  가츠오
19              즐거운맛돈까스
20              마쯔무라돈까스
21                라하노카레
22                   우츠
23                키친우라와
24                  시올돈
25         오리지날분식(이대본점)
26                돈까스정석
27            한성돈까스(별관)
28                   곤돈
29                따봉돈까스
             ...       
170               댓짱돈까스
171                하트타임
172               베테랑분식
173                 묘오또
174       송(현대백화점무역센터점)
175            잉글랜드왕돈까스
176      히바린(신세계백화점강남점)
177              삼백돈돈가츠
178              남산골산채집
179                  연돈
180           댓짱

In [80]:
Naengmyeon_df = get_restaurant_data('냉면')

40 개의 식당 검색 완료.
1 번째 식당 정보 수집중...
2 번째 식당 정보 수집중...
3 번째 식당 정보 수집중...
4 번째 식당 정보 수집중...
5 번째 식당 정보 수집중...
6 번째 식당 정보 수집중...
7 번째 식당 정보 수집중...
8 번째 식당 정보 수집중...
9 번째 식당 정보 수집중...
10 번째 식당 정보 수집중...
11 번째 식당 정보 수집중...
12 번째 식당 정보 수집중...
13 번째 식당 정보 수집중...
14 번째 식당 정보 수집중...
15 번째 식당 정보 수집중...
16 번째 식당 정보 수집중...
17 번째 식당 정보 수집중...
18 번째 식당 정보 수집중...
19 번째 식당 정보 수집중...
20 번째 식당 정보 수집중...
21 번째 식당 정보 수집중...
22 번째 식당 정보 수집중...
23 번째 식당 정보 수집중...
24 번째 식당 정보 수집중...
25 번째 식당 정보 수집중...
26 번째 식당 정보 수집중...
27 번째 식당 정보 수집중...
28 번째 식당 정보 수집중...
29 번째 식당 정보 수집중...
30 번째 식당 정보 수집중...
31 번째 식당 정보 수집중...
32 번째 식당 정보 수집중...
33 번째 식당 정보 수집중...
34 번째 식당 정보 수집중...
35 번째 식당 정보 수집중...
36 번째 식당 정보 수집중...
37 번째 식당 정보 수집중...
38 번째 식당 정보 수집중...
39 번째 식당 정보 수집중...
40 번째 식당 정보 수집중...
냉면 맛집리스트가 완성되었습니다.
        음식점  망고평점 네이버평점 네이버리뷰수 다이닝코드평점 다이닝코드별점  \
0  허브족발(본점)   4.6   N/A    N/A      67     4.3   
1       윤밀원   4.6   N/A    N/A      67     4.7   
2      김진목삼   4.6  4.69    248      55     4.9   
3    진미평양냉면   

In [81]:
Curry_df = get_restaurant_data('카레')

40 개의 식당 검색 완료.
1 번째 식당 정보 수집중...
2 번째 식당 정보 수집중...
3 번째 식당 정보 수집중...
4 번째 식당 정보 수집중...
5 번째 식당 정보 수집중...
6 번째 식당 정보 수집중...
7 번째 식당 정보 수집중...
8 번째 식당 정보 수집중...
9 번째 식당 정보 수집중...
10 번째 식당 정보 수집중...
11 번째 식당 정보 수집중...
12 번째 식당 정보 수집중...
13 번째 식당 정보 수집중...
14 번째 식당 정보 수집중...
15 번째 식당 정보 수집중...
16 번째 식당 정보 수집중...
17 번째 식당 정보 수집중...
18 번째 식당 정보 수집중...
19 번째 식당 정보 수집중...
20 번째 식당 정보 수집중...
21 번째 식당 정보 수집중...
22 번째 식당 정보 수집중...
23 번째 식당 정보 수집중...
24 번째 식당 정보 수집중...
25 번째 식당 정보 수집중...
26 번째 식당 정보 수집중...
27 번째 식당 정보 수집중...
28 번째 식당 정보 수집중...
29 번째 식당 정보 수집중...
30 번째 식당 정보 수집중...
31 번째 식당 정보 수집중...
32 번째 식당 정보 수집중...
33 번째 식당 정보 수집중...
34 번째 식당 정보 수집중...
35 번째 식당 정보 수집중...
36 번째 식당 정보 수집중...
37 번째 식당 정보 수집중...
38 번째 식당 정보 수집중...
39 번째 식당 정보 수집중...
40 번째 식당 정보 수집중...
카레 맛집리스트가 완성되었습니다.
               음식점 망고평점 네이버평점 네이버리뷰수 다이닝코드평점 다이닝코드별점  \
0              마루심  4.7   4.4    776      73     4.1   
1             우미노미  4.7   N/A    N/A      52     4.6   
2               카레  4.7   N/A    N/A      34  

In [12]:
#상호명으로 검색하여 티스토리 블로그 글 추출하는 function
def get_tistory_blog_list(restaurant):    
    
    #검색 편의성을 위해 상호명에서 특수문자 제거
    regex = "\(.*\)|\s-\s.*"
    query = re.sub(regex, '', restaurant)
    #다음 검색에서 블로그/티스토리 필터 적용
    url = 'https://search.daum.net/search?w=blog&nil_search=btn&DA=STC&enc=utf8&q='+query+'&f=section&SA=tistory'

    session = requests.Session()
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content 
    source = BeautifulSoup(html, 'html.parser')
    
    #검색 결과에서 url만 추출
    blogs = source.find('div',{'id':'blogColl'}).find_all('a',{'class':'f_link_b'})    
    
    url_list = []
    count = 0

    #상위 10개의 url만 따와서 리스트에 추가
    for blog in blogs:
        if count < 10:
            url_list.append(blog['href'])
            count += 1

    #url 리스트 return
    return url_list

In [13]:
#티스토리 url 주소로 접속하여 본문 글만 추출하는 function
def get_tistory_content(url):

    session = requests.Session()
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content 
    source = BeautifulSoup(html, 'html.parser')
    
    #다양한 경우의 본문 태그 class 값 고려
    try:
        content = source.find('article',{'id':'content'}).text.replace('\n','')
    except:
        content = ''
    
    if content == '':
        try:
            content = source.find('div',{'class':'article-view'}).text.replace('\n','')
        except:
            content = ''
    
    if content == '':
        try:
            content = source.find('div',{'class':'area_view'}).text.replace('\n','')
        except:
            content = ''
    
    if content == '':
        try:
            content = source.find('div',{'class':'article'}).text.replace('\n','')
        except:
            content = ''
            
    if content == '':
        try:
            content = source.find('div',{'class':'jb-article'}).text.replace('\n','')
        except:
            content = ''
            
    if content == '':
        try:
            content = source.find('div',{'class':'article_cont'}).text.replace('\n','')
        except:
            content = ''
    
    return content
    


In [14]:
#상호명으로 검색하여 티스토리 블로그 리뷰글 text로 5개 가져오는 fucntion
def get_tistory_reviews(restaurant):

    tistory_reviews = []


    count = 0
    success = 0
    blog_list = get_tistory_blog_list(restaurant)

    for blog_url in blog_list:
        #5개만 추출하거나, 10개의 url 모두 시도해도 에러날 경우 포기.
        while success < 5 and count < 11:
            content = get_tistory_content(blog_url)
            count += 1
            if content != '':
                success += 1
                tistory_reviews.append(content)
                #진행상황 알리미
                print(restaurant,'에 대한',len(blog_list),'개의 블로그 중',count,'번째 블로그 크롤링 성공! ',success,'개의 리뷰 수집 완료.')


            else:
                print(restaurant,'에 대한',len(blog_list),'개의 블로그 중',count,'번째 블로그 크롤링 실패.')



    return tistory_reviews



In [20]:
pork_cutlet_tistory = []

for restaurant in pork_cutlet_df['음식점']:
    pork_cutlet_tistory.append(get_tistory_reviews(restaurant))
    

pork_cutlet_df['티스토리리뷰'] = pork_cutlet_tistory
pork_cutlet_df.to_excel('돈까스맛집리스트_{}.xlsx'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')


시키카츠 에 대한 10 개의 블로그 중 1 번째 블로그 크롤링 성공!  1 개의 리뷰 수집 완료.
시키카츠 에 대한 10 개의 블로그 중 2 번째 블로그 크롤링 성공!  2 개의 리뷰 수집 완료.
시키카츠 에 대한 10 개의 블로그 중 3 번째 블로그 크롤링 성공!  3 개의 리뷰 수집 완료.
시키카츠 에 대한 10 개의 블로그 중 4 번째 블로그 크롤링 성공!  4 개의 리뷰 수집 완료.
시키카츠 에 대한 10 개의 블로그 중 5 번째 블로그 크롤링 성공!  5 개의 리뷰 수집 완료.
카와카츠 에 대한 10 개의 블로그 중 1 번째 블로그 크롤링 성공!  1 개의 리뷰 수집 완료.
카와카츠 에 대한 10 개의 블로그 중 2 번째 블로그 크롤링 성공!  2 개의 리뷰 수집 완료.
카와카츠 에 대한 10 개의 블로그 중 3 번째 블로그 크롤링 성공!  3 개의 리뷰 수집 완료.
카와카츠 에 대한 10 개의 블로그 중 4 번째 블로그 크롤링 성공!  4 개의 리뷰 수집 완료.
카와카츠 에 대한 10 개의 블로그 중 5 번째 블로그 크롤링 성공!  5 개의 리뷰 수집 완료.
오제제 에 대한 10 개의 블로그 중 1 번째 블로그 크롤링 성공!  1 개의 리뷰 수집 완료.
오제제 에 대한 10 개의 블로그 중 2 번째 블로그 크롤링 성공!  2 개의 리뷰 수집 완료.
오제제 에 대한 10 개의 블로그 중 3 번째 블로그 크롤링 성공!  3 개의 리뷰 수집 완료.
오제제 에 대한 10 개의 블로그 중 4 번째 블로그 크롤링 성공!  4 개의 리뷰 수집 완료.
오제제 에 대한 10 개의 블로그 중 5 번째 블로그 크롤링 성공!  5 개의 리뷰 수집 완료.
카츠바이콘반 에 대한 10 개의 블로그 중 1 번째 블로그 크롤링 성공!  1 개의 리뷰 수집 완료.
카츠바이콘반 에 대한 10 개의 블로그 중 2 번째 블로그 크롤링 성공!  2 개의 리뷰 수집 완료.
카츠바이콘반 에 대한 10 개의 블로그 중 3 번째 블로그 크롤링 성공!  3 개의 리뷰 수집 완료.
카츠바이콘반 에 

돈까스정석 에 대한 10 개의 블로그 중 1 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 2 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 3 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 4 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 5 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 6 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 7 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 8 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 9 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 10 번째 블로그 크롤링 실패.
돈까스정석 에 대한 10 개의 블로그 중 11 번째 블로그 크롤링 실패.
한성돈까스(별관) 에 대한 10 개의 블로그 중 1 번째 블로그 크롤링 성공!  1 개의 리뷰 수집 완료.
한성돈까스(별관) 에 대한 10 개의 블로그 중 2 번째 블로그 크롤링 성공!  2 개의 리뷰 수집 완료.
한성돈까스(별관) 에 대한 10 개의 블로그 중 3 번째 블로그 크롤링 성공!  3 개의 리뷰 수집 완료.
한성돈까스(별관) 에 대한 10 개의 블로그 중 4 번째 블로그 크롤링 성공!  4 개의 리뷰 수집 완료.
한성돈까스(별관) 에 대한 10 개의 블로그 중 5 번째 블로그 크롤링 성공!  5 개의 리뷰 수집 완료.
곤돈 에 대한 10 개의 블로그 중 1 번째 블로그 크롤링 성공!  1 개의 리뷰 수집 완료.
곤돈 에 대한 10 개의 블로그 중 2 번째 블로그 크롤링 성공!  2 개의 리뷰 수집 완료.
곤돈 에 대한 10 개의 블로그 중 3 번째 블로그 크롤링 성공!  3 개의 리뷰 수집 완료.
곤돈 에 대한 10 개의 블로그 중 4 번째 블로그 크롤링 성공!  4 개의 리뷰 수집 완료.
곤돈 에 대한 10 개의 블로그 중 5 번째 블로그 크롤링 성공!  5 개의 리뷰 

In [3]:
pork_cutlet_df = pd.read_excel('돈까스맛집리스트_210622_0924.xlsx',encoding = 'utf-8')

In [4]:
pork_cutlet_df.head()

,음식점,망고평점,네이버평점,네이버리뷰수,다이닝코드평점,다이닝코드별점,다이닝코드리뷰,주소,전화,종류,가격,티스토리리뷰
0,시키카츠,4.7,NaN,NaN,NaN,NaN,NaN,서울시 동대문구 회기동 60-147,02-6081-8753,까스요리,만원-2만원,"["" 반응형(adsbygoogle = window.adsbygoogle || [])..."
1,카와카츠,4.6,4.43,425.0,60.0,4.4,"[""◈ 맛집기록 #9 '카와카츠'(5.0점 / 5점 만점)- 유명한 프리미엄 돈가스...",서울시 마포구 서교동 465-1 1F 102호,070-8801-2053,까스요리,만원-2만원,"["" 728x90 반응형(adsbygoogle = window.adsbygoogle..."
2,오제제,4.6,NaN,NaN,43.0,4.6,['최근 몇년간 먹은 중에 우동과 돈까스를 가장 맛있게 먹은 곳. 가격이 별로 신경...,서울시 용산구 동자동 43-22 1F,070-4647-4650,라멘/소바/우동,만원 미만,['일식/돈카츠[서울역맛집] 오제제 - 용산 돈카츠 신흥강자!by 비어벨2021. ...
3,카츠바이콘반,4.5,NaN,NaN,59.0,4.9,['11시 30분 오픈! 토요일이어서그런지 11시 전부터 대기 4팀이나 있드라구요....,서울시 강남구 신사동 647-22,02-547-3903,까스요리,만원-2만원,"[""내돈내산 찐맛집/동네 밖 맛집카츠바이콘반 (신사 돈까스 맛집) Multipote..."
4,후니도니(르메이에르종로타운점),4.5,NaN,NaN,59.0,4.5,['유명한이유가 있어요 치즈돈까스와 모밀이 단짠단짠마냥 맛있게느끼하고 개운하고 바삭...,서울시 종로구 종로1가 24 르메이에르종로타운 B1,02-722-5402,까스요리,만원 미만,[]
